In [1]:
import logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(name)s - %(levelname)s -- l.%(lineno)d: %(message)s",
)

In [2]:
import pandas as pd

from src.newsletters.main import runner as runner_newsletters
from src.scoring.main import runner as runner_scoring
from src.reporting.report import Report

2025-02-01 23:20:03,428 - httpx - INFO -- l.1027: HTTP Request: GET https://raw.githubusercontent.com/BerriAI/litellm/main/model_prices_and_context_window.json "HTTP/1.1 200 OK"


In [3]:
report_date_range = ['2024-12-01', '2024-12-03']
news_stories = runner_newsletters(after=report_date_range[0], before=report_date_range[1])
df_news_stories, target_fields = runner_scoring(news_stories)

2025-02-01 23:20:04,128 - googleapiclient.discovery_cache - INFO -- l.49: file_cache is only supported with oauth2client<4.0.0
2025-02-01 23:20:04,132 - src.newsletters.main - INFO -- l.31: Query emails
2025-02-01 23:20:04,939 - src.newsletters.main - INFO -- l.37: Using 1 sender(s): dict_keys(['TLDR AI <dan@tldrnewsletter.com>']).
2025-02-01 23:20:04,940 - src.newsletters.main - INFO -- l.39: Found 2 emails from sender TLDR AI <dan@tldrnewsletter.com>
2025-02-01 23:20:04,940 - src.newsletters.main - INFO -- l.42: Parsing emails to extract news stories
2025-02-01 23:20:12,306 - src.newsletters.main - INFO -- l.49: Newsletters block complete. Found 25 news stories in total.
2025-02-01 23:20:12,307 - src.scoring.main - INFO -- l.24: Converting all news stories to a structured format.
2025-02-01 23:20:12,893 - src.scoring.main - INFO -- l.28: Querying the metadata tags from Google Sheets.
2025-02-01 23:20:12,897 - googleapiclient.discovery_cache - INFO -- l.49: file_cache is only supporte

In [4]:
report = Report(
    df_scored_news_stories=df_news_stories, 
    target_fields=target_fields, 
    report_date_range=report_date_range,
    score_col="score_category_count",
)

2025-02-01 23:20:24,405 - src.reporting.report - INFO -- l.58: Will generate report for range 2024-12-01 to 2024-12-03
2025-02-01 23:20:24,409 - src.reporting.report - INFO -- l.168: Target fields were successfully verified.
2025-02-01 23:20:24,411 - src.reporting.report - INFO -- l.137: We'll use the news stories provided by the dataframe.
2025-02-01 23:20:24,414 - src.reporting.report - INFO -- l.72: Found 25 candidate news stories for this report.


In [5]:
news_stories_for_report = report._filtered_news_stories(min_score_threshold=3, min_nb_entries=5, min_pct_entries=0.1)
news_stories_for_report.keys()

dict_keys(['COMPETTIVE_INTELLIGENCE', 'FUNDING', 'EVALUATION', 'THEMES-OpenAI', 'THEMES-Google', 'THEMES-Agents', 'THEMES-Other'])

In [6]:
report_str = report._create_report_from_news_stories(news_stories_for_report)

2025-02-01 23:20:24,455 - src.genai_model.genai_model - INFO -- l.92: Found 13 models for model_type large
2025-02-01 23:20:24,457 - src.genai_model.genai_model - INFO -- l.95: List of models included: ['gemini/gemini-exp-1206', 'openrouter/google/gemini-exp-1206:free', 'gemini/gemini-exp-1121', 'openrouter/google/gemini-exp-1121:free', 'gemini/gemini-exp-1114', 'openrouter/google/gemini-exp-1114:free', 'gemini/gemini-1.5-pro-002', 'gemini/gemini-1.5-pro-exp-0827', 'openrouter/meta-llama/llama-3.1-405b-instruct:free', 'gemini/gemini-1.5-pro-001', 'mistral/mistral-large-2411', 'mistral/mistral-large-2407', 'mistral/mistral-large-2402']
23:20:24 - LiteLLM:INFO: utils.py:2752 - 
LiteLLM completion() model= gemini-exp-1206; provider = gemini
2025-02-01 23:20:24,485 - LiteLLM - INFO -- l.2752: 
LiteLLM completion() model= gemini-exp-1206; provider = gemini
2025-02-01 23:20:30,319 - httpx - INFO -- l.1027: HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-exp-

In [7]:
with open("test.txt", "w") as f:
    f.write(report_str)

In [8]:
report.df_ns

,title,url,news_provider,source_of_the_news,text,news_summary,date_source,competitive_intelligence,themes,market_intelligence,personalities,score
0,ELON MUSK SEEKS TO BLOCK OPENAI'S FOR-PROFIT C...,https://www.theverge.com/2024/11/30/24309697/e...,www.theverge.com,TLDR AI <dan@tldrnewsletter.com>,,Elon Musk's legal team has filed a motion to p...,"Mon, 2 Dec 2024 14:29:42 +0000",[],[Legal],[OpenAI],[],2
1,PERPLEXITY MULLS GETTING INTO HARDWARE,https://techcrunch.com/2024/11/26/perplexity-m...,techcrunch.com,TLDR AI <dan@tldrnewsletter.com>,,"Perplexity's CEO plans to develop a ""simple, u...","Mon, 2 Dec 2024 14:29:42 +0000",[],"[AI&GenAI, Multimodal]",[],[],2
2,INFLECTION AI CEO SAYS IT'S DONE TRYING TO MAK...,https://techcrunch.com/2024/11/26/inflection-c...,techcrunch.com,TLDR AI <dan@tldrnewsletter.com>,,Inflection AI shifted its focus from developin...,"Mon, 2 Dec 2024 14:29:42 +0000",[],"[AI&GenAI, Model]",[],[],2
3,INTELLECT-1 RELEASE: THE FIRST GLOBALLY TRAINE...,https://www.primeintellect.ai/blog/intellect-1...,www.primeintellect.ai,TLDR AI <dan@tldrnewsletter.com>,,INTELLECT-1 is a 10B parameter model trained o...,"Mon, 2 Dec 2024 14:29:42 +0000",[],"[Model Training, Evaluation]",[],[],2
4,DETECT AND LEARN UNSEEN OBJECTS,https://arxiv.org/abs/2411.18207v1,arxiv.org,TLDR AI <dan@tldrnewsletter.com>,,This new framework pushes object detection int...,"Mon, 2 Dec 2024 14:29:42 +0000",[],"[AI&GenAI, Multimodal]",[],[],2
5,MAKING UNDERWATER IMAGES CLEAR,https://arxiv.org/abs/2411.18296v1,arxiv.org,TLDR AI <dan@tldrnewsletter.com>,,HUPE is an AI-powered method that improves und...,"Mon, 2 Dec 2024 14:29:42 +0000",[],"[AI&GenAI, Multimodal]",[],[],2
6,MAPPING THE IONOSPHERE WITH THE POWER OF ANDROID,https://research.google/blog/mapping-the-ionos...,research.google,TLDR AI <dan@tldrnewsletter.com>,,Google researchers were able to accurately map...,"Mon, 2 Dec 2024 14:29:42 +0000",[],[],[Google],[],0
7,INTRODUCING LTNTORCH,https://arxiv.org/abs/2409.16045v1,arxiv.org,TLDR AI <dan@tldrnewsletter.com>,,Logic Tensor Networks (LTN) merge deep learnin...,"Mon, 2 Dec 2024 14:29:42 +0000",[],[ML&DL],[],[],1
8,REFINING PRETRAINING DATA PROGRAMMATICALLY,https://gair-nlp.github.io/ProX/homepage.html,gair-nlp.github.io,TLDR AI <dan@tldrnewsletter.com>,,ProX is a framework that treats data refinemen...,"Mon, 2 Dec 2024 14:29:42 +0000",[],"[Model, Model Training]",[],[],2
9,CHATS WITH VIDEOS IN REAL TIME,https://huggingface.co/wangyueqian/MMDuet,huggingface.co,TLDR AI <dan@tldrnewsletter.com>,,"MMDuet is a novel ""video-text duet"" interactio...","Mon, 2 Dec 2024 14:29:42 +0000",[],[AI&GenAI],[],[],1


In [9]:
df_selected = report.df_ns[report.df_ns["score"] >= 2]
print(f"Selected {len(df_selected)} out of {len(df_news_stories)}")

Selected 12 out of 15


In [10]:
df_selected

,title,url,news_provider,source_of_the_news,text,news_summary,date_source,competitive_intelligence,themes,market_intelligence,personalities,score
0,ELON MUSK SEEKS TO BLOCK OPENAI'S FOR-PROFIT C...,https://www.theverge.com/2024/11/30/24309697/e...,www.theverge.com,TLDR AI <dan@tldrnewsletter.com>,,Elon Musk's legal team has filed a motion to p...,"Mon, 2 Dec 2024 14:29:42 +0000",[],[Legal],[OpenAI],[],2
1,PERPLEXITY MULLS GETTING INTO HARDWARE,https://techcrunch.com/2024/11/26/perplexity-m...,techcrunch.com,TLDR AI <dan@tldrnewsletter.com>,,"Perplexity's CEO plans to develop a ""simple, u...","Mon, 2 Dec 2024 14:29:42 +0000",[],"[AI&GenAI, Multimodal]",[],[],2
2,INFLECTION AI CEO SAYS IT'S DONE TRYING TO MAK...,https://techcrunch.com/2024/11/26/inflection-c...,techcrunch.com,TLDR AI <dan@tldrnewsletter.com>,,Inflection AI shifted its focus from developin...,"Mon, 2 Dec 2024 14:29:42 +0000",[],"[AI&GenAI, Model]",[],[],2
3,INTELLECT-1 RELEASE: THE FIRST GLOBALLY TRAINE...,https://www.primeintellect.ai/blog/intellect-1...,www.primeintellect.ai,TLDR AI <dan@tldrnewsletter.com>,,INTELLECT-1 is a 10B parameter model trained o...,"Mon, 2 Dec 2024 14:29:42 +0000",[],"[Model Training, Evaluation]",[],[],2
4,DETECT AND LEARN UNSEEN OBJECTS,https://arxiv.org/abs/2411.18207v1,arxiv.org,TLDR AI <dan@tldrnewsletter.com>,,This new framework pushes object detection int...,"Mon, 2 Dec 2024 14:29:42 +0000",[],"[AI&GenAI, Multimodal]",[],[],2
5,MAKING UNDERWATER IMAGES CLEAR,https://arxiv.org/abs/2411.18296v1,arxiv.org,TLDR AI <dan@tldrnewsletter.com>,,HUPE is an AI-powered method that improves und...,"Mon, 2 Dec 2024 14:29:42 +0000",[],"[AI&GenAI, Multimodal]",[],[],2
8,REFINING PRETRAINING DATA PROGRAMMATICALLY,https://gair-nlp.github.io/ProX/homepage.html,gair-nlp.github.io,TLDR AI <dan@tldrnewsletter.com>,,ProX is a framework that treats data refinemen...,"Mon, 2 Dec 2024 14:29:42 +0000",[],"[Model, Model Training]",[],[],2
10,BUILDING LLMS IS PROBABLY NOT GOING BE A BRILL...,https://calpaterson.com/porter.html,calpaterson.com,TLDR AI <dan@tldrnewsletter.com>,,LLM makers like OpenAI face significant challe...,"Mon, 2 Dec 2024 14:29:42 +0000",[],"[AI&GenAI, Model, Funding]","[OpenAI, NVIDIA]",[],5
11,ROX,https://www.notboring.co/p/rox,www.notboring.co,TLDR AI <dan@tldrnewsletter.com>,,"A new AI-native CRM, Rox, aims to disrupt Sale...","Mon, 2 Dec 2024 14:29:42 +0000",[],"[AI&GenAI, Funding]",[],[],2
12,PLAYAI RAISES $21M FUNDING AND RELEASES A NEW ...,https://blog.play.ai/blog/21m-funding,blog.play.ai,TLDR AI <dan@tldrnewsletter.com>,,PlayAI raised $21 Million to advance voice-fir...,"Mon, 2 Dec 2024 14:29:42 +0000",[],"[AI&GenAI, Multimodal, Funding]",[],[],3


In [12]:
report.get_path_to_report_log(extension="parquet")

PosixPath('log/log_report_2024-12-01_2024-12-03_20250102235331.parquet')